In [ ]:
import xarray as xr
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import scienceplots
from tqdm import tqdm
import cmcrameri.cm as cmc
import cartopy.crs as ccrs

plt.style.use(['science'])

In [ ]:
fpath = Path(r"C:\Users\mathi\OneDrive\Dokumenter\DTU\Kandidat\Syntese\AltimeterGridding\Grids\high_res_grids_v7")
datas = []
for file in fpath.glob("2014_9_28.nc"):
    datas.append(xr.open_dataset(file))
data = xr.concat(datas, dim="time")
data = data.sortby("time")

In [ ]:
mss21_path = Path(r"C:\Users\mathi\Danmarks Tekniske Universitet\Casper Tygesen Bang-Hansen - ProcessedGrids\mss_DTU21-15_fullres.nc")
mss21 = xr.load_dataarray(mss21_path)

In [ ]:
mss21_downsampled = mss21.coarsen(Longitude=10).mean().coarsen(Latitude=10).mean()

In [ ]:
lat_idx = (mss21_downsampled.lat[:,0] <= -25) & (mss21_downsampled.lat[:,0] >= -45)
lon_idx = (mss21_downsampled.lon[0]+180 <= 45) & (mss21_downsampled.lon[0]+180 >= 5)
mss21_area = mss21_downsampled.sel(Latitude = lat_idx, Longitude = lon_idx)

In [ ]:
plt.figure(figsize=(13,6))
plt.imshow(mss21,vmin=-.1, vmax=.1,cmap=cmc.vik)
plt.colorbar()
plt.show()

In [ ]:
data["sla21"] = data.sla - mss21_area.values[::-1]

In [ ]:
cmap=cmc.vik
vlim = 0.75

title_size = 20
label_size = 14
tick_size = 12

# extent = [data.Longitude.min().item(), data.Longitude.max().item(), data.Latitude.min().item(), data.Latitude.max().item()]
extent = [5,45,-45,-25]

fig = plt.figure(figsize=(13,6), dpi=500)
ax = plt.axes(projection=ccrs.PlateCarree())
im=ax.imshow(
    data.sla21[0], 
    origin="lower", 
    extent=extent, 
    vmin=-vlim, 
    vmax=vlim, 
    cmap=cmap,
    transform=ccrs.PlateCarree()
)
ax.set_xlabel(f"Longitude [\N{DEGREE SIGN}]", fontsize=label_size)
ax.set_ylabel(f"Latitude [\N{DEGREE SIGN}]", fontsize=label_size)
ax.coastlines()
# ax.set_extent(extent)
ax.set_xticks([5, 15, 25, 35, 45])
ax.set_yticks([-45, -40, -35, -30, -25])
ax.tick_params(labelsize=tick_size)
cbar = fig.colorbar(im)
cbar.set_label("SLA [cm]", fontsize=label_size)
cbar.ax.tick_params(labelsize=tick_size)
title_text = f"{data.time[0].values.astype('datetime64[ns]').astype('datetime64[D]')}"
ax.set_title(title_text, fontsize=title_size)
plt.show()

In [ ]:
data_poor_res = Path(r"C:\Users\mathi\Danmarks Tekniske Universitet\Casper Tygesen Bang-Hansen - ProcessedGrids", "without_polar_v6_mss21.nc")
data_poor_all = xr.open_dataset(data_poor_res, engine="netcdf4")
data_poor = data_poor_all.sel(time = (data_poor_all.time == np.datetime64("2014-09-28T12:00:00.000000000")))

lat_idxp = (data_poor.Latitude[:,0] <= -25) & (data_poor.Latitude[:,0] >= -45)
lon_idxp = (data_poor.Longitude[0] <= 45) & (data_poor.Longitude[0] >= 5)
data_poor_area = data_poor.sel(lats = lat_idxp, lons = lon_idxp)

In [ ]:
cmap=cmc.vik
vlim = 0.75

title_size = 20
label_size = 14
tick_size = 12

# extent = [data.Longitude.min().item(), data.Longitude.max().item(), data.Latitude.min().item(), data.Latitude.max().item()]
extent = [5,45,-45,-25]

fig = plt.figure(figsize=(13,6), dpi=500)
ax = plt.axes(projection=ccrs.PlateCarree())
im=ax.imshow(
    data_poor_area.sla[0], 
    origin="lower", 
    extent=extent, 
    vmin=-vlim, 
    vmax=vlim, 
    cmap=cmap,
    transform=ccrs.PlateCarree()
)
ax.set_xlabel(f"Longitude [\N{DEGREE SIGN}]", fontsize=label_size)
ax.set_ylabel(f"Latitude [\N{DEGREE SIGN}]", fontsize=label_size)
ax.coastlines()
# ax.set_extent(extent)
ax.set_xticks([5, 15, 25, 35, 45])
ax.set_yticks([-45, -40, -35, -30, -25])
ax.tick_params(labelsize=tick_size)
cbar = fig.colorbar(im)
cbar.set_label("SLA [cm]", fontsize=label_size)
cbar.ax.tick_params(labelsize=tick_size)
title_text = f"{data.time[0].values.astype('datetime64[ns]').astype('datetime64[D]')}"
ax.set_title(title_text, fontsize=title_size)
plt.show()

In [ ]:
cmap=cmc.vik
vlim = 0.1

title_size = 20
label_size = 14
tick_size = 12

# extent = [data.Longitude.min().item(), data.Longitude.max().item(), data.Latitude.min().item(), data.Latitude.max().item()]
extent = [5,45,-45,-25]

fig = plt.figure(figsize=(13,6), dpi=500)
ax = plt.axes(projection=ccrs.PlateCarree())
im=ax.imshow(
    mss21_area, 
    extent=extent, 
    vmin=-vlim, 
    vmax=vlim, 
    cmap=cmap,
    transform=ccrs.PlateCarree()
)
ax.set_xlabel(f"Longitude [\N{DEGREE SIGN}]", fontsize=label_size)
ax.set_ylabel(f"Latitude [\N{DEGREE SIGN}]", fontsize=label_size)
ax.coastlines()
# ax.set_extent(extent)
ax.set_xticks([5, 15, 25, 35, 45])
ax.set_yticks([-45, -40, -35, -30, -25])
ax.tick_params(labelsize=tick_size)
cbar = fig.colorbar(im)
cbar.set_label("SLA [cm]", fontsize=label_size)
cbar.ax.tick_params(labelsize=tick_size)
title_text = f"{data.time[0].values.astype('datetime64[ns]').astype('datetime64[D]')}"
ax.set_title(title_text, fontsize=title_size)
plt.show()